In [25]:
from atj_trading.backtester import get_ohlc_history, Backtester
from datetime import datetime, timedelta
import MetaTrader5 as mt5
from IPython.display import display
import plotly.express as px

In [5]:
# connect to MT5
mt5.initialize()

True

In [6]:
symbol = 'BTCUSD'
timeframe = mt5.TIMEFRAME_D1
start_dt = datetime.now() - timedelta(days=365 * 10)
end_dt = datetime.now()

ohlc_df = get_ohlc_history(symbol, timeframe, start_dt, end_dt)
ohlc_df

,time,open,high,low,close
0,2014-09-07,473.57,476.99,472.27,474.85
1,2014-09-08,474.85,475.98,457.10,468.40
2,2014-09-09,468.99,469.97,456.00,462.56
3,2014-09-10,462.56,480.00,462.00,470.47
4,2014-09-11,470.47,475.70,464.00,469.50
...,...,...,...,...,...
3057,2024-08-30,59471.90,59940.98,57678.04,59018.72
3058,2024-08-31,58995.66,59439.36,58721.46,58933.41
3059,2024-09-01,58908.48,59116.96,57111.53,58389.93
3060,2024-09-02,58390.25,59220.13,57092.05,59016.32


In [23]:
# ToDo: Analyze Price Patterns by the time of the year
ohlc_df['gain'] = ohlc_df['open'].shift(-1) - ohlc_df['open']
ohlc_df['perc_gain'] = (ohlc_df['gain'] / ohlc_df['open']) * 100

ohlc_df['month'] = ohlc_df['time'].dt.month
ohlc_df['year'] = ohlc_df['time'].dt.year
ohlc_df

,time,open,high,low,close,month,gain,year,perc_gain
0,2014-09-07,473.57,476.99,472.27,474.85,9,1.28,2014,0.270287
1,2014-09-08,474.85,475.98,457.10,468.40,9,-5.86,2014,-1.234074
2,2014-09-09,468.99,469.97,456.00,462.56,9,-6.43,2014,-1.371031
3,2014-09-10,462.56,480.00,462.00,470.47,9,7.91,2014,1.710048
4,2014-09-11,470.47,475.70,464.00,469.50,9,-0.97,2014,-0.206177
...,...,...,...,...,...,...,...,...,...
3057,2024-08-30,59471.90,59940.98,57678.04,59018.72,8,-476.24,2024,-0.800782
3058,2024-08-31,58995.66,59439.36,58721.46,58933.41,8,-87.18,2024,-0.147774
3059,2024-09-01,58908.48,59116.96,57111.53,58389.93,9,-518.23,2024,-0.879721
3060,2024-09-02,58390.25,59220.13,57092.05,59016.32,9,680.65,2024,1.165691


In [24]:
gain_by_month = ohlc_df.groupby(['year', 'month'], as_index=False)['perc_gain'].sum()
display(gain_by_month)

fig = px.bar(gain_by_month, x='month', y='perc_gain', title='Gain by month', color='year')
fig

,year,month,perc_gain
0,2014,9,-21.138579
1,2014,10,-13.014514
2,2014,11,16.365666
3,2014,12,-17.613447
4,2015,1,-21.839331
...,...,...,...
116,2024,5,12.875070
117,2024,6,-8.109839
118,2024,7,4.973627
119,2024,8,-7.527145


In [28]:
# create trade logic
def on_bar(data, trades, orders):
    volume = round(10000 / data['open'], 2)
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['month'] in [2, 10] and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]
        
        if data['month'] in [3, 11]:
            orders.close_trade(trade)

In [29]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -20

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,BTCUSD,buy,26.5,2014-10-01 00:00:00,377.29,2014-11-02 00:00:00,321.86,0,0,{},-1468.90,-530.0,-1998.9,-1998.9,8001.1
1,closed,BTCUSD,buy,44.58,2015-02-01 00:00:00,224.3,2015-03-01 00:00:00,250.0,0,0,{},1145.71,-891.6,254.11,-1744.79,8255.21
2,closed,BTCUSD,buy,42.9,2015-10-01 00:00:00,233.08,2015-11-01 00:00:00,308.1,0,0,{},3218.36,-858.0,2360.36,615.57,10615.57
3,closed,BTCUSD,buy,26.93,2016-02-01 00:00:00,371.27,2016-03-01 00:00:00,432.79,0,0,{},1656.73,-538.6,1118.13,1733.7,11733.7
4,closed,BTCUSD,buy,16.44,2016-10-02 00:00:00,608.29,2016-11-01 00:00:00,689.0,0,0,{},1326.87,-328.8,998.07,2731.77,12731.77
5,closed,BTCUSD,buy,10.58,2017-02-01 00:00:00,945.2,2017-03-01 00:00:00,1174.5,0,0,{},2425.99,-211.6,2214.39,4946.16,14946.16
6,closed,BTCUSD,buy,2.31,2017-10-02 00:00:00,4325.37,2017-11-01 00:00:00,6348.22,0,0,{},4672.78,-46.2,4626.58,9572.74,19572.74
7,closed,BTCUSD,buy,1.01,2018-02-01 00:00:00,9907.22,2018-03-01 00:00:00,10480.99,0,0,{},579.51,-20.2,559.31,10132.05,20132.05
8,closed,BTCUSD,buy,1.53,2018-10-01 00:00:00,6536.11,2018-11-01 00:00:00,6287.58,0,0,{},-380.25,-30.6,-410.85,9721.2,19721.2
9,closed,BTCUSD,buy,2.95,2019-02-01 00:00:00,3393.57,2019-03-01 00:00:00,3778.79,0,0,{},1136.40,-59.0,1077.4,10798.6,20798.6


In [30]:
pnl_chart = bt.plot_pnl()
pnl_chart